In [1]:
from os import getcwd
from pathlib import Path

import flopy as fp
import numpy as np

In [2]:
workspace = Path(getcwd(), "model")
workspace

WindowsPath('c:/Users/buckl/Documents/Github/flopy_mf6_work/notebooks/implementing/gwf_twri01/model')

In [3]:
sim_name = "gwf_twri01"
sim = fp.mf6.MFSimulation(
    sim_name=sim_name,
    version="mf6",
    exe_name=Path("C:\\Users\\buckl\\Documents\\Software\\mf6.4.1\\bin\\mf6.exe"),
    sim_ws=workspace
)
sim

sim_name = gwf_twri01
sim_path = c:\Users\buckl\Documents\Github\flopy_mf6_work\notebooks\implementing\gwf_twri01\model
exe_name = C:\Users\buckl\Documents\Software\mf6.4.1\bin\mf6.exe

###################
Package mfsim.nam
###################

package_name = mfsim.nam
filename = mfsim.nam
package_type = nam
model_or_simulation_package = simulation
simulation_name = gwf_twri01



In [4]:
fp.mf6.ModflowTdis(
    simulation=sim,
    time_units="seconds",
    nper=1,
    perioddata=[(86400.0, 1, 1.0)],
    filename=f"{sim_name}.tdis",
    pname="tdis"
)

package_name = tdis
filename = gwf_twri01.tdis
package_type = tdis
model_or_simulation_package = simulation
simulation_name = gwf_twri01

Block options
--------------------
time_units
{internal}
('seconds')


Block dimensions
--------------------
nper
{internal}
(1)


Block perioddata
--------------------
perioddata
{internal}
(rec.array([(86400., 1, 1.)],
          dtype=[('perlen', '<f8'), ('nstp', '<i4'), ('tsmult', '<f8')]))



In [5]:
fp.mf6.ModflowIms(
    simulation=sim,
    outer_dvclose=1e-9,
    outer_maximum=50,
    inner_maximum=100,
    inner_dvclose=1e-9,
    rcloserecord=[(1e-6, "strict")],
    filename=f"{sim_name}.ims",
    pname="ims"
)

package_name = ims
filename = gwf_twri01.ims
package_type = ims
model_or_simulation_package = simulation
simulation_name = gwf_twri01

Block nonlinear
--------------------
outer_dvclose
{internal}
(1e-09)

outer_maximum
{internal}
(50)


Block linear
--------------------
inner_maximum
{internal}
(100)

inner_dvclose
{internal}
(1e-09)

rcloserecord
{internal}
(rec.array([('inner_rclose', 1.e-06, 'strict')],
          dtype=[('inner_rclose_label', 'O'), ('inner_rclose', '<f8'), ('rclose_option', 'O')]))



In [6]:
model = fp.mf6.ModflowGwf(
    simulation=sim,
    modelname=sim_name,
    model_nam_file=f"{sim_name}.nam"
)
model

name = gwf_twri01
model_type = gwf6
version = mf6
model_relative_path = .


In [7]:
fp.mf6.ModflowGwfdis(
    model=model,
    length_units="FEET",
    nlay=5,
    nrow=15,
    ncol=15,
    delr=5000.0,
    delc=5000.0,
    top=200.0,
    botm=[-150.0, -200.0, -300.0, -350.0, -450.0],
    filename=f"{sim_name}.dis",
    pname="dis",
)

package_name = dis
filename = gwf_twri01.dis
package_type = dis
model_or_simulation_package = model
model_name = gwf_twri01

Block options
--------------------
length_units
{internal}
('feet')


Block dimensions
--------------------
nlay
{internal}
(5)

nrow
{internal}
(15)

ncol
{internal}
(15)


Block griddata
--------------------
delr
{constant 5000.0}

delc
{constant 5000.0}

top
{constant 200.0}

botm
Layer_1{constant -150.0}
Layer_2{constant -200.0}
Layer_3{constant -300.0}
Layer_4{constant -350.0}
Layer_5{constant -450.0}



In [8]:
k = [0.001, 1e-8, 0.0001, 5e-7, 0.0002]
fp.mf6.ModflowGwfnpf(
    model=model,
    cvoptions=[(True, "DEWATERED")],
    perched=True,
    save_specific_discharge=True,
    icelltype=[1, 0, 0, 0, 0],
    k=k,
    k33=k,
    filename=f"{sim_name}.npf",
    pname="npf",
)

package_name = npf
filename = gwf_twri01.npf
package_type = npf
model_or_simulation_package = model
model_name = gwf_twri01

Block options
--------------------
cvoptions
{internal}
([(True, 'DEWATERED')])

perched
{internal}
(True)

save_specific_discharge
{internal}
(True)


Block griddata
--------------------
icelltype
Layer_1{constant 1}
Layer_2{constant 0}
Layer_3{constant 0}
Layer_4{constant 0}
Layer_5{constant 0}

k
Layer_1{constant 0.001}
Layer_2{constant 1e-08}
Layer_3{constant 0.0001}
Layer_4{constant 5e-07}
Layer_5{constant 0.0002}

k33
Layer_1{constant 0.001}
Layer_2{constant 1e-08}
Layer_3{constant 0.0001}
Layer_4{constant 5e-07}
Layer_5{constant 0.0002}



In [9]:
fp.mf6.ModflowGwfic(
    model=model,
    strt=0.0,
    filename=f"{sim_name}.ic",
    pname="ic"
)

package_name = ic
filename = gwf_twri01.ic
package_type = ic
model_or_simulation_package = model
model_name = gwf_twri01

Block griddata
--------------------
strt
{constant 0.0}



In [18]:
def make_chd_iter():
    for layer in [0, 2]:
        for row in range(15):
            yield ((layer, row, 0), 0.0)

fp.mf6.ModflowGwfchd(
    model=model,
    maxbound=30,
    stress_period_data=list(make_chd_iter()),
    filename=f"{sim_name}.chd",
    pname="chd_0"
)

package_name = chd_0
filename = gwf_twri01.chd
package_type = chd
model_or_simulation_package = model
model_name = gwf_twri01

Block dimensions
--------------------
maxbound
{internal}
(30)


Block period
--------------------
stress_period_data
{0: <flopy.mf6.data.mfdataplist.PandasListStorage object at 0x000002E0F6566C10>}


In [19]:
model.package_dict['chd_0'].stress_period_data.get_data(key=0)

rec.array([((0, 0, 0), 0.), ((0, 1, 0), 0.), ((0, 2, 0), 0.),
           ((0, 3, 0), 0.), ((0, 4, 0), 0.), ((0, 5, 0), 0.),
           ((0, 6, 0), 0.), ((0, 7, 0), 0.), ((0, 8, 0), 0.),
           ((0, 9, 0), 0.), ((0, 10, 0), 0.), ((0, 11, 0), 0.),
           ((0, 12, 0), 0.), ((0, 13, 0), 0.), ((0, 14, 0), 0.),
           ((2, 0, 0), 0.), ((2, 1, 0), 0.), ((2, 2, 0), 0.),
           ((2, 3, 0), 0.), ((2, 4, 0), 0.), ((2, 5, 0), 0.),
           ((2, 6, 0), 0.), ((2, 7, 0), 0.), ((2, 8, 0), 0.),
           ((2, 9, 0), 0.), ((2, 10, 0), 0.), ((2, 11, 0), 0.),
           ((2, 12, 0), 0.), ((2, 13, 0), 0.), ((2, 14, 0), 0.)],
          dtype=[('cellid', 'O'), ('head', '<f8')])

In [26]:
def make_drn_iter():
    elev = [0, 0, 10, 20, 30, 50, 70, 90, 100]
    for col in range(1, 10):
        yield ((0, 7, col), float(elev[col - 1]), 1.0)

fp.mf6.ModflowGwfdrn(
    model=model,
    maxbound=9,
    stress_period_data=list(make_drn_iter()),
    filename=f"{sim_name}.drn",
    pname="drn_0"
)

package_name = drn_0
filename = gwf_twri01.drn
package_type = drn
model_or_simulation_package = model
model_name = gwf_twri01

Block dimensions
--------------------
maxbound
{internal}
(9)


Block period
--------------------
stress_period_data
{0: <flopy.mf6.data.mfdataplist.PandasListStorage object at 0x000002E0F7ADC610>}


In [27]:
model.package_dict['drn_0'].stress_period_data.get_data(key=0)

rec.array([((0, 7, 1),   0., 1.), ((0, 7, 2),   0., 1.),
           ((0, 7, 3),  10., 1.), ((0, 7, 4),  20., 1.),
           ((0, 7, 5),  30., 1.), ((0, 7, 6),  50., 1.),
           ((0, 7, 7),  70., 1.), ((0, 7, 8),  90., 1.),
           ((0, 7, 9), 100., 1.)],
          dtype=[('cellid', 'O'), ('elev', '<f8'), ('cond', '<f8')])

In [29]:
def make_wel_iter():
    # wells in unconfined aquifer
    for row in range(8, 13, 2):
        for col in range(7, 14, 2):
            yield ((0, row, col), -5.0)

    # wells in middle confined aquifer
    yield ((2, 3, 5), -5.0)
    yield ((2, 5, 11), -5.0)

    # well in lower confined aquifer
    yield ((4, 4, 10), -5.0)

fp.mf6.ModflowGwfwel(
    model=model,
    maxbound=15,
    stress_period_data=list(make_wel_iter()),
    filename=f"{sim_name}.wel",
    pname="wel_0"
)

package_name = wel_0
filename = gwf_twri01.wel
package_type = wel
model_or_simulation_package = model
model_name = gwf_twri01

Block dimensions
--------------------
maxbound
{internal}
(15)


Block period
--------------------
stress_period_data
{0: <flopy.mf6.data.mfdataplist.PandasListStorage object at 0x000002E0F6533A90>}


In [30]:
model.package_dict['wel_0'].stress_period_data.get_data(key=0)

rec.array([((0, 8, 7), -5.), ((0, 8, 9), -5.), ((0, 8, 11), -5.),
           ((0, 8, 13), -5.), ((0, 10, 7), -5.), ((0, 10, 9), -5.),
           ((0, 10, 11), -5.), ((0, 10, 13), -5.), ((0, 12, 7), -5.),
           ((0, 12, 9), -5.), ((0, 12, 11), -5.), ((0, 12, 13), -5.),
           ((2, 3, 5), -5.), ((2, 5, 11), -5.), ((4, 4, 10), -5.)],
          dtype=[('cellid', 'O'), ('q', '<f8')])

In [31]:
fp.mf6.ModflowGwfrcha(
    model=model,
    recharge=3e-8,
    filename=f"{sim_name}.rcha",
    pname="rcha_0"
)

package_name = rcha_0
filename = gwf_twri01.rcha
package_type = rcha
model_or_simulation_package = model
model_name = gwf_twri01

Block options
--------------------
readasarrays
{internal}
(True)


Block period
--------------------
irch
None
recharge
{constant 3e-08}

aux
None


In [32]:
fp.mf6.ModflowGwfoc(
    model=model,
    budget_filerecord=[f"{sim_name}.cbc"],
    head_filerecord=[f"{sim_name}.hds"],
    saverecord=[
        ("head", "all"),
        ("budget", "all")
    ],
    filename=f"{sim_name}.oc",
    pname="oc"
)

package_name = oc
filename = gwf_twri01.oc
package_type = oc
model_or_simulation_package = model
model_name = gwf_twri01

Block options
--------------------
budget_filerecord
{internal}
(rec.array([('gwf_twri01.cbc',)],
          dtype=[('budgetfile', 'O')]))

head_filerecord
{internal}
(rec.array([('gwf_twri01.hds',)],
          dtype=[('headfile', 'O')]))


Block period
--------------------
saverecord
{internal}
(rec.array([('head', 'all', None), ('budget', 'all', None)],
          dtype=[('rtype', 'O'), ('ocsetting', 'O'), ('ocsetting_data', 'O')]))

printrecord
None


In [33]:
sim.check()

Checking model "gwf_twri01"...

gwf_twri01 MODEL DATA VALIDATION SUMMARY:
  No errors or warnings encountered.

  Checks that passed:
    npf package: zero or negative horizontal hydraulic conductivity values
    npf package: vertical hydraulic conductivity values below checker threshold of 1e-11
    npf package: vertical hydraulic conductivity values above checker threshold of 100000.0
    npf package: horizontal hydraulic conductivity values below checker threshold of 1e-11
    npf package: horizontal hydraulic conductivity values above checker threshold of 100000.0
    chd_0 package: BC indices valid
    chd_0 package: not a number (Nan) entries
    chd_0 package: BC in inactive cells
    drn_0 package: BC indices valid
    drn_0 package: not a number (Nan) entries
    drn_0 package: BC in inactive cells
    wel_0 package: BC indices valid
    wel_0 package: not a number (Nan) entries
    wel_0 package: BC in inactive cells

Checking for missing simulation packages...


In [34]:
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package ims...
  writing model gwf_twri01...
    writing model name file...
    writing package dis...
    writing package npf...
    writing package ic...
    writing package chd_0...
    writing package drn_0...
    writing package wel_0...
    writing package rcha_0...
    writing package oc...


In [41]:
success, buff = sim.run_simulation()
assert success, "MODFLOW did not terminate normally!"

FloPy is using the following executable to run the model: ..\..\..\..\..\..\Software\mf6.4.1\bin\mf6.exe
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                        VERSION 6.4.1 Release 12/09/2022

   MODFLOW 6 compiled Dec 09 2022 18:18:37 with Intel(R) Fortran Intel(R) 64
   Compiler Classic for applications running on Intel(R) 64, Version 2021.7.0
                             Build 20220726_000000

This software has been approved for release by the U.S. Geological 
Survey (USGS). Although the software has been subjected to rigorous 
review, the USGS reserves the right to update the software as needed 
pursuant to further analysis and review. No warranty, expressed or 
implied, is made by the USGS or the U.S. Government as to the 
functionality of the software and related material nor shall the 
fact of release constitute any such warranty. Furthermore, the 
software is released on condition that neither the US

In [42]:
sim.delete_output_files()